In [ ]:
# RAG System - Interactive UI with ipywidgets

This notebook provides a complete web-like interface using ipywidgets that replicates the three-column layout:
- **Left**: Chat Interface
- **Middle**: AI Assistant (Suggestions + Summary)  
- **Right**: Knowledge Base


In [ ]:
# Import required libraries
import os
import json
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from datetime import datetime
from dotenv import load_dotenv
from rag_system import RAGSystem

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully")
print(f"🔑 OpenAI API Key configured: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")

# Custom CSS for better styling
display(HTML("""
<style>
.chat-message {
    padding: 8px 12px;
    margin: 4px 0;
    border-radius: 8px;
    max-width: 90%;
}
.customer-message {
    background-color: #e3f2fd;
    margin-left: auto;
    text-align: right;
}
.agent-message {
    background-color: #f3e5f5;
    margin-right: auto;
}
.system-message {
    background-color: #fff3e0;
    text-align: center;
    font-style: italic;
}
.suggestion-button {
    margin: 2px;
    padding: 4px 8px;
    border: 1px solid #ddd;
    border-radius: 4px;
    background-color: #f8f9fa;
    cursor: pointer;
}
.suggestion-button:hover {
    background-color: #e9ecef;
}
</style>
"""))


In [ ]:
# Initialize RAG System
print("🚀 Initializing RAG System...")

try:
    rag_system = RAGSystem(
        similarity_top_k=5,
        persist_dir='./storage',
        use_reranker=True,
        conversation_context_length=5
    )
    print("✅ RAG System initialized successfully!")
    
    # Get system status
    status = rag_system.get_system_status()
    print(f"📚 {status['document_count']} documents loaded")
    
except Exception as e:
    print(f"❌ Error initializing RAG System: {e}")
    rag_system = None

# Global conversation storage
conversation_history = []
current_suggestions = []
current_summary = ""


In [ ]:
# Create UI Components

# Chat Panel (Left)
chat_output = widgets.Output(layout=widgets.Layout(height='400px', overflow='auto'))
chat_input = widgets.Textarea(
    placeholder="Type your message here...",
    layout=widgets.Layout(width='100%', height='60px')
)
send_button = widgets.Button(
    description="Send",
    button_style='primary',
    layout=widgets.Layout(width='100px')
)
clear_chat_button = widgets.Button(
    description="Clear Chat",
    button_style='warning',
    layout=widgets.Layout(width='100px')
)

chat_controls = widgets.HBox([send_button, clear_chat_button])
chat_panel = widgets.VBox([
    widgets.HTML("<h3>💬 Chat Interface</h3>"),
    chat_output,
    chat_input,
    chat_controls
], layout=widgets.Layout(width='32%', padding='10px', border='1px solid #ddd'))

# AI Assistant Panel (Middle)
summary_output = widgets.HTML(
    value="<div style='padding: 10px; background: #f8f9fa; border-radius: 5px; margin: 5px 0;'>Conversation summary will appear here...</div>",
    layout=widgets.Layout(height='80px')
)
suggestions_output = widgets.Output(layout=widgets.Layout(height='300px', overflow='auto'))

ai_panel = widgets.VBox([
    widgets.HTML("<h3>🤖 AI Assistant</h3>"),
    widgets.HTML("<h4>📋 Summary</h4>"),
    summary_output,
    widgets.HTML("<h4>💡 Suggested Responses</h4>"),
    suggestions_output
], layout=widgets.Layout(width='36%', padding='10px', border='1px solid #ddd'))

# Knowledge Base Panel (Right)
kb_output = widgets.Output(layout=widgets.Layout(height='450px', overflow='auto'))

kb_panel = widgets.VBox([
    widgets.HTML("<h3 style='background: linear-gradient(135deg, #4caf50 0%, #45a049 100%); color: white; padding: 10px; margin: -10px -10px 10px -10px; border-radius: 5px 5px 0 0;'>📚 Knowledge Base</h3>"),
    kb_output
], layout=widgets.Layout(width='32%', padding='10px', border='1px solid #ddd'))

# Main UI Layout
main_ui = widgets.HBox([chat_panel, ai_panel, kb_panel], 
                      layout=widgets.Layout(width='100%'))

print("✅ UI Components created")


In [ ]:
# UI Helper Functions

def add_chat_message(sender, message, sender_type="customer"):
    """Add a message to the chat display."""
    timestamp = datetime.now().strftime("%H:%M:%S")
    
    if sender_type == "customer":
        css_class = "customer-message"
        sender_label = "You"
    elif sender_type == "agent":
        css_class = "agent-message"
        sender_label = "Agent"
    else:
        css_class = "system-message"
        sender_label = "System"
    
    with chat_output:
        display(HTML(f"""
        <div class="chat-message {css_class}">
            <strong>{sender_label}</strong> <small>[{timestamp}]</small><br>
            {message.replace(chr(10), '<br>')}
        </div>
        """))
    
    # Scroll to bottom
    chat_output.layout.height = chat_output.layout.height

def update_summary(summary_text):
    """Update the summary display."""
    if summary_text:
        summary_output.value = f"<div style='padding: 10px; background: #e8f5e8; border-radius: 5px; margin: 5px 0; border-left: 4px solid #4caf50;'>{summary_text}</div>"
    else:
        summary_output.value = "<div style='padding: 10px; background: #f8f9fa; border-radius: 5px; margin: 5px 0;'>Conversation summary will appear here...</div>"

def create_suggestion_button(suggestion_text, index):
    """Create a button for a suggestion that can be used."""
    button = widgets.Button(
        description=f"Use Response {index + 1}",
        button_style='info',
        layout=widgets.Layout(width='140px', margin='2px')
    )
    
    def on_use_suggestion(b):
        # Add suggestion as agent response
        add_chat_message("Agent", suggestion_text, "agent")
        conversation_history.append({
            'sender': 'agent',
            'message': suggestion_text,
            'timestamp': datetime.now().isoformat()
        })
        
        # Show confirmation
        with suggestions_output:
            print(f"✅ Used suggestion {index + 1}")
    
    button.on_click(on_use_suggestion)
    return button

def update_suggestions(suggestions_list):
    """Update the suggestions display."""
    global current_suggestions
    current_suggestions = suggestions_list
    
    with suggestions_output:
        clear_output()
        if suggestions_list:
            for i, suggestion in enumerate(suggestions_list):
                # Create button for each suggestion
                button = create_suggestion_button(suggestion, i)
                display(button)
                
                # Display suggestion text
                display(HTML(f"""
                <div style='padding: 8px; margin: 5px 0; background: #f0f8ff; border-radius: 5px; border-left: 3px solid #2196f3;'>
                    <strong>Suggestion {i + 1}:</strong><br>
                    {suggestion}
                </div>
                """))
        else:
            print("No suggestions available")

def update_knowledge_base():
    """Update the knowledge base display."""
    if not rag_system:
        with kb_output:
            clear_output()
            print("❌ RAG System not initialized")
        return
    
    status = rag_system.get_system_status()
    
    with kb_output:
        clear_output()
        print(f"📊 System Status:")
        print(f"• Documents: {status['document_count']}")
        print(f"• Vector Store: {status['vector_store_type']}")
        print(f"• Reranker: {'Enabled' if status['use_reranker'] else 'Disabled'}")
        print(f"• Top-K: {status['similarity_top_k']}")
        
        print(f"\n📚 Available Documents:")
        for i, filename in enumerate(status['document_files'], 1):
            print(f"{i:2d}. {filename}")

print("✅ Helper functions created")


In [ ]:
# Event Handlers

def process_customer_message(message_text):
    """Process a customer message and update the UI."""
    if not rag_system or not message_text.strip():
        return
    
    # Add customer message to chat and history
    add_chat_message("Customer", message_text, "customer")
    conversation_history.append({
        'sender': 'customer',
        'message': message_text,
        'timestamp': datetime.now().isoformat()
    })
    
    # Show processing message
    add_chat_message("System", "🔄 Processing your message...", "system")
    
    try:
        # Get AI suggestions and summary
        result = rag_system.get_suggestions(message_text, conversation_history[:-1], 'customer')
        
        # Update summary
        summary = result.get('summary', '')
        update_summary(summary)
        
        # Update suggestions
        suggestions = result.get('suggestions', [])
        update_suggestions(suggestions)
        
        # Remove processing message and show completion
        add_chat_message("System", "✅ AI suggestions generated!", "system")
        
    except Exception as e:
        add_chat_message("System", f"❌ Error: {str(e)}", "system")
        print(f"Error processing message: {e}")

def on_send_click(b):
    """Handle send button click."""
    message = chat_input.value.strip()
    if message:
        process_customer_message(message)
        chat_input.value = ""  # Clear input

def on_clear_chat_click(b):
    """Handle clear chat button click."""
    global conversation_history
    conversation_history = []
    
    with chat_output:
        clear_output()
    
    # Reset other panels
    update_summary("")
    update_suggestions([])
    
    add_chat_message("System", "Chat cleared. Start a new conversation!", "system")

def on_enter_key(change):
    """Handle Enter key in chat input (Ctrl+Enter to send)."""
    if change['name'] == 'value':
        # Note: ipywidgets doesn't easily support Ctrl+Enter, but we can add a note
        pass

# Attach event handlers
send_button.on_click(on_send_click)
clear_chat_button.on_click(on_clear_chat_click)

print("✅ Event handlers attached")


In [ ]:
# Initialize and Display the UI

# Initialize knowledge base display
update_knowledge_base()

# Add welcome message
add_chat_message("System", "👋 Welcome to the RAG System! Type a message and click Send to get AI-powered suggestions.", "system")

# Display the main UI
display(main_ui)

print("\n🎉 RAG System UI is ready!")
print("💡 Tips:")
print("  • Type customer messages in the chat input")
print("  • Click 'Send' to get AI suggestions and conversation summary")
print("  • Use the 'Use Response X' buttons to add suggestions to the chat")
print("  • Click 'Clear Chat' to start over")


In [ ]:
## Additional Utilities

Run the cells below for extra functionality:


In [ ]:
# Quick Test Functions

def quick_test_scenario(scenario_name):
    """Run a predefined test scenario."""
    scenarios = {
        "greeting": "Hello, I need help with ordering exotic meats",
        "product": "I'm interested in kangaroo steaks for a dinner party",
        "shipping": "How long does shipping take and what are the costs?",
        "complaint": "I received the wrong order and need to return it",
        "food_safety": "What are your food safety certifications?"
    }
    
    if scenario_name in scenarios:
        message = scenarios[scenario_name]
        chat_input.value = message
        print(f"✅ Loaded scenario '{scenario_name}': {message}")
        print("Click the Send button to process this message")
    else:
        print(f"❌ Unknown scenario. Available: {list(scenarios.keys())}")

# Test buttons
test_buttons = []
scenarios = ["greeting", "product", "shipping", "complaint", "food_safety"]

for scenario in scenarios:
    button = widgets.Button(description=f"Test: {scenario.title()}", button_style='success')
    def make_handler(s):
        return lambda b: quick_test_scenario(s)
    button.on_click(make_handler(scenario))
    test_buttons.append(button)

print("🧪 Quick Test Functions:")
display(widgets.HBox(test_buttons[:3]))
display(widgets.HBox(test_buttons[3:]))


In [ ]:
# Conversation Analysis and Export

def export_conversation():
    """Export the current conversation to JSON."""
    if not conversation_history:
        print("❌ No conversation to export")
        return
    
    filename = f"conversation_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    export_data = {
        'timestamp': datetime.now().isoformat(),
        'conversation_count': len(conversation_history),
        'conversation': conversation_history,
        'last_summary': current_summary,
        'last_suggestions': current_suggestions
    }
    
    with open(filename, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"✅ Conversation exported to {filename}")

def analyze_conversation():
    """Analyze the current conversation."""
    if not conversation_history:
        print("❌ No conversation to analyze")
        return
    
    customer_msgs = [msg for msg in conversation_history if msg['sender'] == 'customer']
    agent_msgs = [msg for msg in conversation_history if msg['sender'] == 'agent']
    
    print("📊 Conversation Analysis:")
    print(f"  • Total messages: {len(conversation_history)}")
    print(f"  • Customer messages: {len(customer_msgs)}")
    print(f"  • Agent messages: {len(agent_msgs)}")
    
    if customer_msgs:
        avg_customer_length = sum(len(msg['message']) for msg in customer_msgs) / len(customer_msgs)
        print(f"  • Average customer message length: {avg_customer_length:.1f} characters")
    
    if agent_msgs:
        avg_agent_length = sum(len(msg['message']) for msg in agent_msgs) / len(agent_msgs)
        print(f"  • Average agent message length: {avg_agent_length:.1f} characters")

def show_conversation_json():
    """Display the conversation history as JSON."""
    if not conversation_history:
        print("❌ No conversation history")
        return
    
    print("📋 Conversation History (JSON):")
    print(json.dumps(conversation_history, indent=2))

# Utility buttons
export_btn = widgets.Button(description="Export Conversation", button_style='info')
analyze_btn = widgets.Button(description="Analyze Conversation", button_style='info')
json_btn = widgets.Button(description="Show JSON", button_style='info')

export_btn.on_click(lambda b: export_conversation())
analyze_btn.on_click(lambda b: analyze_conversation())
json_btn.on_click(lambda b: show_conversation_json())

print("📁 Conversation Tools:")
display(widgets.HBox([export_btn, analyze_btn, json_btn]))


In [ ]:
---

## 🎯 You're All Set!

The interactive RAG system UI is now running above. This notebook provides:

- **Full three-column layout** (Chat | AI Assistant | Knowledge Base) 
- **Interactive chat interface** with real-time message processing
- **AI-powered suggestions** with "Use This Response" buttons
- **Dynamic conversation summaries** that update as you chat
- **Knowledge base integration** showing available documents
- **Quick test scenarios** for common customer inquiries  
- **Conversation analysis and export tools**

**How to use:**
1. Type customer messages in the chat input on the left
2. Click "Send" to get AI suggestions and summaries in the middle panel
3. Use the "Use Response X" buttons to add suggestions to the chat as agent responses
4. Monitor the knowledge base status on the right
5. Use the test buttons and utilities below for advanced features

The interface works exactly like the web version but runs entirely within Jupyter!
